In [2]:
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit
from sys import stdout

## Simulation setup

In [3]:
prmtop = app.AmberPrmtopFile('BPTI_gas.prmtop')
inpcrd = app.AmberInpcrdFile('BPTI_gas.inpcrd')

system = prmtop.createSystem(nonbondedMethod=app.NoCutoff, constraints=None)
integrator = mm.LangevinIntegrator(298.15*unit.kelvin, 1.0/unit.picoseconds, 
    1.0*unit.femtoseconds)

platform = mm.Platform.getPlatformByName('CPU')
simulation = app.Simulation(prmtop.topology, system, integrator, platform)
simulation.context.setPositions(inpcrd.positions)

## Energy minimization

In [4]:
st = simulation.context.getState(getPositions=True,getEnergy=True)
print("Potential energy before minimization is %s" % st.getPotentialEnergy())

print('Minimizing...')
simulation.minimizeEnergy(maxIterations=100)

st = simulation.context.getState(getPositions=True,getEnergy=True)
print("Potential energy after minimization is %s" % st.getPotentialEnergy())

Potential energy before minimization is -2108.982628950387 kJ/mol
Minimizing...
Potential energy after minimization is -5072.5430810602475 kJ/mol


## Equilibration (brief!)

In [3]:
simulation.context.setVelocitiesToTemperature(298.15*unit.kelvin)
print('Equilibrating...')
simulation.step(1000)
print('Done!')

Equilibrating...


## Production

In [4]:
simulation.reporters.append(app.DCDReporter('BPTI_sim.dcd', 1))
simulation.reporters.append(app.StateDataReporter(stdout, 250, step=True, time=True, 
    potentialEnergy=True, temperature=True, speed=True, separator='\t'))

print('Running Production...')
simulation.step(9000)
print('Done!')

Running Production...
#"Step"	"Time (ps)"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"
1250	1.2499999999999731	-3105.631888955444	269.30286296858617	0
1500	1.4999999999999456	-3129.4944089510127	285.1700845506675	5.23
1750	1.749999999999918	-3252.459043730104	279.4265338392294	5.14
2000	1.9999999999998905	-3153.8408622071292	281.5325712192819	5.2
2250	2.249999999999863	-3007.2937828939957	284.99183385286716	5.2
2500	2.4999999999998357	-3052.353317712761	303.20776391179845	5.21
2750	2.749999999999808	-2895.674894649277	292.2414362351824	5.21
3000	2.9999999999997806	-2975.8202455607516	312.38762810460014	5.21
3250	3.249999999999753	-2916.8055406813164	308.6728350315733	5.18
3500	3.4999999999997256	-2780.1994362252753	304.5095286366434	5.19
3750	3.749999999999698	-2845.204098359449	306.75961821818265	5.19
4000	3.9999999999996705	-2958.01518109268	311.2318949088547	5.21
4250	4.249999999999754	-3114.489981353525	319.8674329677283	5.19
4500	4.4999999999998375	-3144.6643272